In [1]:
from transformers import pipeline
from datasets import load_dataset
from transformers import AutoTokenizer, pipeline
import torch
import numpy as np
from sklearn.metrics import f1_score, accuracy_score, precision_score, recall_score


# Load your dataset
ds = load_dataset("higopires/RePro-categories-multilabel")

def filter_inadequada(example):
    return example["INADEQUADA"] == 0
device = "cuda" if torch.cuda.is_available() else "cpu"
ds = ds.filter(filter_inadequada)
print(f"Using device: {device}")


c:\Users\diogo\AppData\Local\Programs\Python\Python313\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Using device: cuda


In [2]:
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using device: {device}")
classifier = pipeline("zero-shot-classification",mode="PORTULAN/albertina-1b5-portuguese-ptbr-encoder",    return_all_scores=True,device=device) #, model="facebook/bart-large-mnli")

No model was supplied, defaulted to facebook/bart-large-mnli and revision d7645e1 (https://huggingface.co/facebook/bart-large-mnli).
Using a pipeline without specifying a model name and revision in production is not recommended.


Using device: cuda


Device set to use cuda


In [3]:
def compute_metrics(predictions,labels):
    # Calculate sample-wise F1 score
    f1 = f1_score(labels, predictions, average='micro', zero_division=0)
    precision = precision_score(labels, predictions, average='micro', zero_division=0)

    recall = recall_score(labels, predictions, average='micro', zero_division=0)
    return {'f1_micro': float(f1), 'precision': float(precision), 'recall': float(recall)}
print(compute_metrics([[1,0,1],[0,1,0]],[[1,0,1],[0,1,0]]))

{'f1_micro': 1.0, 'precision': 1.0, 'recall': 1.0}


In [ ]:
preds = []
y = []
for i in range(len(ds['test'])):
    if i % 100 == 0:
        print(f"Processing example {i}/{len(ds['test'])}")
    
    example = ds['test'][i]
    text = example["review_text"]
    
    # Perform zero-shot classification
    entrega = classifier(text, candidate_labels=["entrega", "não entrega"], multi_label=True)
    outros = classifier(text, candidate_labels=["outros", "não outros"], multi_label=True)
    produtos = classifier(text, candidate_labels=["produtos", "não produtos"], multi_label=True)
    condições_Recb = classifier(text, candidate_labels=["condições de recebimento", "não condições de recebimento"], multi_label=True)
    anuncio = classifier(text, candidate_labels=["anuncio", "não anuncio"], multi_label=True)
    labels = [entrega['labels'][0], outros['labels'][0], produtos['labels'][0], condições_Recb['labels'][0], anuncio['labels'][0]]
    # transform the labels to a binary format
    labels = [1 if label == "entrega" or label == "outros" or label == "produtos" or label == "condições de recebimento" or label == "anuncio" else 0 for label in labels]
    preds.append(labels)
    y.append([example["ENTREGA"],example["OUTROS"],example["PRODUTO"],example["CONDICOESDERECEBIMENTO"],example["ANUNCIO"]])
    #print(preds,y)
    #print(compute_metrics(preds,y))
print(compute_metrics(preds,y))



Processing example 0/966


You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset


Processing example 100/966
Processing example 200/966
Processing example 300/966
Processing example 400/966
Processing example 500/966
